# Spark

Apache Spark — это мощная и гибкая система для обработки больших объемов данных, предлагающая широкий спектр возможностей для аналитики и машинного обучения. 

In [1]:
import os
import socket
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col, desc, rank, row_number

In [2]:
print('user:', os.environ['JUPYTERHUB_SERVICE_PREFIX'])

def uiWebUrl(self):
    from urllib.parse import urlparse
    web_url = self._jsc.sc().uiWebUrl().get()
    port = urlparse(web_url).port
    return '{}proxy/{}/jobs/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)

SparkContext.uiWebUrl = property(uiWebUrl)

conf = SparkConf().set('spark.master', 'local[*]').set('spark.driver.memory', '4g')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

user: /user/notkguin/


In [3]:
cd ~

/home/jovyan


In [4]:
sdf = spark.read.csv(
    'aggrigation_logs_per_week.csv', 
    sep=',', 
    header=True
)
sdf.printSchema()

root
 |-- courseid: string (nullable = true)
 |-- userid: string (nullable = true)
 |-- num_week: string (nullable = true)
 |-- s_all: string (nullable = true)
 |-- s_all_avg: string (nullable = true)
 |-- s_course_viewed: string (nullable = true)
 |-- s_course_viewed_avg: string (nullable = true)
 |-- s_q_attempt_viewed: string (nullable = true)
 |-- s_q_attempt_viewed_avg: string (nullable = true)
 |-- s_a_course_module_viewed: string (nullable = true)
 |-- s_a_course_module_viewed_avg: string (nullable = true)
 |-- s_a_submission_status_viewed: string (nullable = true)
 |-- s_a_submission_status_viewed_avg: string (nullable = true)
 |-- NameR_Level: string (nullable = true)
 |-- Name_vAtt: string (nullable = true)
 |-- Depart: string (nullable = true)
 |-- Name_OsnO: string (nullable = true)
 |-- Name_FormOPril: string (nullable = true)
 |-- LevelEd: string (nullable = true)
 |-- Num_Sem: string (nullable = true)
 |-- Kurs: string (nullable = true)
 |-- Date_vAtt: string (nullable =

Выведем  первые пять записей из Spark DataFrame

In [5]:
sdf.limit(5).toPandas().head()

,courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,...,s_a_submission_status_viewed_avg,NameR_Level,Name_vAtt,Depart,Name_OsnO,Name_FormOPril,LevelEd,Num_Sem,Kurs,Date_vAtt
0,71262,34527,6,9,9,4,4,0,0,0,...,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
1,71262,34527,7,0,"4,5",0,2,0,0,0,...,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
2,71262,34527,8,0,3,0,"1,3333",0,0,0,...,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
3,71262,34527,9,0,"2,25",0,1,0,0,0,...,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
4,71262,34527,10,0,"1,8",0,"0,8",0,0,0,...,0,3,Экзамен,22,1,1,1,2,2,18.06.2022


In [6]:
print('Количество записей в наборе данных:', sdf.count())

Количество записей в наборе данных: 414528


In [7]:
sdf.orderBy('s_course_viewed').limit(5).toPandas()

,courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,...,s_a_submission_status_viewed_avg,NameR_Level,Name_vAtt,Depart,Name_OsnO,Name_FormOPril,LevelEd,Num_Sem,Kurs,Date_vAtt
0,84236,33622,25,0,"23,1",0,"4,2",0,"3,05",0,...,"3,75",4,Экзамен,23,1,1,1,2,2,23.06.2022
1,71262,34527,10,0,"1,8",0,"0,8",0,0,0,...,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
2,75656,29359,28,0,"0,2174",0,"0,1304",0,0,0,...,"0,0435",2,Экзамен,12,1,1,2,4,3,17.06.2022
3,71262,34527,12,0,"1,4286",0,"0,7143",0,0,0,...,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
4,84236,33622,26,0,22,0,4,0,"2,9048",0,...,"3,5714",4,Экзамен,23,1,1,1,2,2,23.06.2022


In [8]:
sdf.select('Name_vAtt').distinct().show()

+---------+
|Name_vAtt|
+---------+
|  Экзамен|
+---------+



Группировка данных, подсчет частоты появления

In [9]:
sdf.groupby('userid').count().show()

+------+-----+
|userid|count|
+------+-----+
| 35444|   72|
| 34657|   96|
| 29089|   48|
| 29573|   48|
| 30966|   24|
| 35350|   96|
| 29539|   72|
| 21783|   72|
| 24269|   72|
| 22121|   48|
|  5613|   48|
| 24528|   48|
| 27492|   48|
| 15539|   24|
| 24078|   24|
| 25350|   48|
| 30054|   24|
| 35438|   72|
| 27753|   72|
| 27563|   96|
+------+-----+
only showing top 20 rows



In [10]:
sdf.groupby('userid').count().sort(col('count').desc()).show()

+------+-----+
|userid|count|
+------+-----+
| 33470|  120|
| 33541|  120|
| 33528|  120|
| 20252|  120|
| 24347|  120|
| 33864|  120|
| 20734|  120|
| 24631|  120|
| 34345|  120|
| 25123|  120|
| 25267|  120|
| 30067|  120|
| 34186|  120|
| 33583|  120|
| 36019|  120|
| 36001|  120|
| 24710|  120|
| 33685|  120|
| 33463|  120|
| 35787|  120|
+------+-----+
only showing top 20 rows



Фильтрация данных

In [11]:
sdf.select(
    'userid',
    'num_week',
    'courseid',
    's_course_viewed'
).filter(
    (col('userid') == '33470') & (col('courseid') == '74257')
).limit(
    30
).toPandas()

,userid,num_week,courseid,s_course_viewed
0,33470,6,74257,5
1,33470,7,74257,0
2,33470,8,74257,12
3,33470,9,74257,4
4,33470,10,74257,0
5,33470,11,74257,14
6,33470,12,74257,5
7,33470,13,74257,5
8,33470,14,74257,2
9,33470,15,74257,8


## Задание 1:
### Анализ активности студентов на портале
Рассчитать общее количество событий (s_all) и просмотров курсов (s_course_viewed) по каждой неделе (num_week).

Примечание. Используйте методы groupBy + agg. 

Проанализируйте полученные данные, какие выводы можно сделать?

In [12]:
weekly_activity = sdf.groupBy('num_week').agg(
    F.sum('s_all').alias('total_events'),
    F.sum('s_course_viewed').alias('total_course_views')
).orderBy('num_week')

weekly_activity.show()

+--------+------------+------------------+
|num_week|total_events|total_course_views|
+--------+------------+------------------+
|      10|    141778.0|           33449.0|
|      11|    173100.0|           39493.0|
|      12|    162041.0|           37901.0|
|      13|    172962.0|           38890.0|
|      14|    170305.0|           37556.0|
|      15|    178829.0|           39322.0|
|      16|    177650.0|           38385.0|
|      17|    181629.0|           37328.0|
|      18|    167044.0|           34461.0|
|      19|    191952.0|           40742.0|
|      20|    249532.0|           52905.0|
|      21|    319598.0|           64802.0|
|      22|    388401.0|           72299.0|
|      23|    384501.0|           70501.0|
|      24|    397465.0|           69962.0|
|      25|    314633.0|           53815.0|
|      26|    148590.0|           27531.0|
|      27|     21641.0|            4210.0|
|      28|      1554.0|             467.0|
|      29|       945.0|             308.0|
+--------+-

Наблюдается выраженная сезонность активности:

* Наибольшая активность приходится на недели 20-24 (вероятно, конец семестра/сессия)
* Пик активности - 24 неделя
* Резкий спад после 25 недели (вероятно, начало каникул)
* Просмотры курсов составляют примерно 17-20% от общего числа событий, что показывает значительную долю других активностей (тесты, задания и т.д.)
* На неделях 28-29 активность минимальна, что может указывать на межсеместровый период.

## Задание 2.
### Определение самых популярных курсов

Найти топ-5 курсов (courseid) по среднему количеству просмотров (s_course_viewed_avg).


In [13]:
top_courses = sdf.groupBy('courseid').agg(
    F.avg('s_course_viewed').alias('avg_course_views')
).orderBy(F.col('avg_course_views').desc()).limit(5)

top_courses.show()

+--------+------------------+
|courseid|  avg_course_views|
+--------+------------------+
|   76419|26.473577235772357|
|   78733|17.930555555555557|
|   78705|16.133333333333333|
|   82552|14.083333333333334|
|   73823| 13.90798611111111|
+--------+------------------+



## Задание 3: 
### Мини-исследование

Определить: имеется ли связь между количеством просмотров студентов отдельных курсов и количеством студентов, изучающих отдельный курс.



In [14]:
course_stats = sdf.groupBy('courseid').agg(
    F.avg('s_course_viewed').alias('avg_views'),
    F.countDistinct('userid').alias('student_count')
)

course_stats.corr('avg_views', 'student_count')

0.08089185664017717

## Задание 4. 
### Сравнение активности студентов на бюджете и контракте

Найти разницу в среднем количестве всех событий (s_all_avg) между студентами на бюджете и контракте (name_osno). 

Какие выводы можно сделать?



In [15]:
activity_by_funding = sdf.groupBy('Name_OsnO').agg(
    F.avg('s_all_avg').alias('avg_activity')
)

activity_by_funding.show()

+---------+------------------+
|Name_OsnO|      avg_activity|
+---------+------------------+
|        1|  8.41703942142887|
|        2|3.4732525769317584|
+---------+------------------+



Студенты на бюджете проявляют в 2.4 раза большую активность (8.42 vs 3.47), чем контрактники.

## Задание 5.
### Исследование зависимости активности студентов от формы обучения

Определить, как форма обучения (name_formopril) влияет на активность студентов (s_all_avg)?



In [16]:
activity_by_education_form = sdf.groupBy('Name_FormOPril').agg(
    F.avg('s_all_avg').alias('avg_activity')
).orderBy('avg_activity', ascending=False)

activity_by_education_form.show()

+--------------+------------------+
|Name_FormOPril|      avg_activity|
+--------------+------------------+
|             1|11.699871821520128|
|             3|1.2953216374269005|
|             2|1.2438109107202868|
+--------------+------------------+



## Задание 6. 
### Выявление активности студентов по семестрам

Определить, в каком семестре (num_sem) студенты проявляют наибольшую активность (по s_all_avg).



In [17]:
activity_by_semester = sdf.groupBy('Num_Sem').agg(
    F.avg('s_all_avg').alias('avg_activity')
).orderBy('avg_activity', ascending=False)

activity_by_semester.show()

+-------+-------------------+
|Num_Sem|       avg_activity|
+-------+-------------------+
|      2|  9.363531867267612|
|      6|  6.696356437227533|
|      4|  6.216743692230827|
|      8|  3.508361204013378|
|     10|  2.418956814357824|
|     12|0.08901884340480831|
+-------+-------------------+



## Задание 7.
### Определение кафедр с наибольшей активностью студентов

Найти топ-3 кафедры (depart), где студенты наиболее активны (по s_all_avg).



In [18]:
top_departments = sdf.groupBy('Depart').agg(
    F.avg('s_all_avg').alias('avg_activity')
).orderBy(F.col('avg_activity').desc()).limit(3)

top_departments.show()

+------+------------------+
|Depart|      avg_activity|
+------+------------------+
|     4|26.657972928359193|
|    12|20.158597662771285|
|    14| 15.56260819388344|
+------+------------------+



## Задание 8. 
### Анализ успеваемости студентов в зависимости от активности

Найти среднюю оценку (namer_level) для студентов с разной активностью (s_all_avg).



In [19]:
performance_by_activity = sdf.groupBy('NameR_Level').agg(
    F.avg('s_all_avg').alias('avg_activity')
).orderBy('NameR_Level')

performance_by_activity.show()

+-----------+------------------+
|NameR_Level|      avg_activity|
+-----------+------------------+
|          2|3.1998184156767797|
|          3| 2.584847376336219|
|          4|  5.93765992745269|
|          5|11.122604763025123|
+-----------+------------------+



## Задание 9.
### Выявление студентов с аномально низкой активностью
Найти студентов с количеством событий (s_all_avg) ниже среднего по курсу (kurs).

Подсказка: Используйте window functions. 
`````
from pyspark.sql.window import Window
from pyspark.sql.functions import mean
`````

In [20]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col

student_avg = sdf.groupBy("userid", "kurs").agg(
    avg("s_all_avg").alias("student_avg_activity")
)

window = Window.partitionBy("kurs")

result = (student_avg.withColumn("kurs_avg", avg("student_avg_activity").over(window))
          .filter(col("student_avg_activity") < col("kurs_avg"))
          .select("userid", "kurs", "student_avg_activity", "kurs_avg")
          .orderBy("kurs", "userid"))

result.show()

+------+----+--------------------+------------------+
|userid|kurs|student_avg_activity|          kurs_avg|
+------+----+--------------------+------------------+
| 15110|   2|                1.25|12.279773140694227|
| 24342|   2|                 0.0|12.279773140694227|
| 27492|   2| 0.05263157894736842|12.279773140694227|
| 28985|   2|                 0.0|12.279773140694227|
| 29367|   2|                 0.5|12.279773140694227|
| 29370|   2|                 3.8|12.279773140694227|
| 29636|   2| 0.42424242424242425|12.279773140694227|
| 30193|   2|  0.3888888888888889|12.279773140694227|
| 30239|   2|                 0.0|12.279773140694227|
| 31783|   2|                 0.0|12.279773140694227|
| 31896|   2| 0.09090909090909091|12.279773140694227|
| 32160|   2|                 0.0|12.279773140694227|
| 32193|   2|                 0.0|12.279773140694227|
| 32785|   2|                 0.0|12.279773140694227|
| 33393|   2|   6.857142857142857|12.279773140694227|
| 33430|   2|               

## Задание 10.
### Кластеризация студентов по активности 

Разделить студентов на 3 группы (низкая, средняя, высокая активность) с помощью K-Means.

Примечание.  Используйте KMeans из pyspark.ml.clustering. 
Обратите внимание, что по каждому студенту в наборе данных представлены логи, поэтому прежде чем проводить кластеризацию нужно сначала  провести агрегацию данных для каждого студента.

In [23]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

student_activity = sdf.groupBy('userid').agg(
    F.avg('s_all_avg').alias('avg_activity'),
    F.avg('s_course_viewed_avg').alias('avg_course_views'),
    F.avg('s_q_attempt_viewed_avg').alias('avg_quiz_views')
)

assembler = VectorAssembler(
    inputCols=['avg_activity', 'avg_course_views', 'avg_quiz_views'],
    outputCol='features'
)
data_for_clustering = assembler.transform(student_activity)

kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(data_for_clustering)

clustered_data = model.transform(data_for_clustering)
clustered_data.groupBy('prediction').count().show()

cluster_centers = model.clusterCenters()

for i, center in enumerate(cluster_centers):
    print(f"Кластер {i}:")
    print(f"  Средняя активность (avg_activity): {center[0]:.2f}")

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  180|
|         2| 1366|
|         0| 4770|
+----------+-----+

Кластер 0:
  Средняя активность (avg_activity): 3.10
Кластер 1:
  Средняя активность (avg_activity): 72.35
Кластер 2:
  Средняя активность (avg_activity): 25.02
